### Functions to read files

In [6]:
def readImgNames(train=True):
    names = list()
    loc = "train_images/*" if train else "test_images/*"
    return [i[i.index("\\")+1:] for i in glob.glob(loc)]

In [5]:
def readConfigs():
    df = pd.read_csv("train.csv")
    return df

In [6]:
def readImg(name):
    return cv2.imread(f"train_images/{name}")

### Functions to make a submission

In [2]:
def make_mask_from_rle(rle, width=256, height=1600):
    res = np.array([0 for _ in range(width * height )])
    for inx in range(0,len(rle),2):
        start = rle[inx]-1
        end = start + rle[inx+1]
        res[start:end] = 1
    return np.flipud( np.rot90( res.reshape(height,width), k=1 ) )

In [1]:
def make_rle(ar): # must be numpy array
    ar = ar.flatten()
    new_ar = list()
    
    flag = False
    for inx,val in enumerate(ar):
        if not val:
            if not flag:
                new_ar.append(inx+1) 
                flag = True
        else:
            if flag:
                flag = False
                new_ar.append(inx-new_ar[-1]+1)
    if flag:
        new_ar.append(ar.shape[0] - new_ar[-1] + 1)
        
    return new_ar  

#### Resize function

In [5]:
def resize(train=True):
    path = "train_images/*" if train else "test_images/*" 
    for img in glob.glob(path):
        im = cv2.imread(img)
        im = cv2.resize(im,(256,256))
        name = img[img.index('\\')+1:]
        if train:
            cv2.imwrite(f"resized/{name}", im)
        else:
            cv2.imwrite(f"resized_test/{name}", im)


__Score function__

In [1]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [2]:
def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score
def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)